<a href="https://colab.research.google.com/github/byunsy/bioinformatics-algorithms-py/blob/main/BA_Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salmonella enterica Problem

### Function

In [10]:
def MinimumSkew(genome):
    skew = 0
    min  = 0
    min_indices = []

    for i, base in enumerate(genome):

        # update skew value based on base-nucleotides
        if base == "G":
            skew += 1
        elif base == "C":
            skew -= 1

        # if new minimum, update min and create new min_indices
        if skew < min:
            min = skew
            min_indices = [i+1]
        elif skew == min:
            min_indices.append(i+1) # append to min_indices if skew value is min 

    return min_indices

In [1]:
def ReverseComplement(pattern):
    comp = ""
    
    # Get complement
    for char in pattern:
        if char == 'A':
            comp += 'T'
        elif char == 'T':
            comp += 'A'
        elif char == 'G':
            comp += 'C'
        elif char == 'C':
            comp += 'G'

    # Reverse it and return
    return comp[::-1]

In [2]:
def HammingDistance(str1, str2):
    if len(str1) != len(str2):
        raise Exception("Error: Two strings must be of equal length.")

    mismatch = 0
    for i in range(len(str1)):
        if str1[i] != str2[i]:
            mismatch += 1
    return mismatch

In [3]:
def ApproxPatternCount(pattern, text, d):
    count = 0
    bound = len(text) - len(pattern) + 1
    for i in range(bound):
        pattern2 = text[i:i+len(pattern)] 
        if HammingDistance(pattern, pattern2) <= d:
            count += 1
    return count

In [4]:
def Neighbors(pattern, d):
    if d == 0:
        return [pattern]
    if len(pattern) == 1:
        return ['A', 'C', 'G', 'T']

    first  = pattern[0]
    suffix = pattern[1:]
    nucleotides = ['A', 'C', 'G', 'T']
    neighborhood = []
    suffix_neighbors = Neighbors(suffix, d)

    for text in suffix_neighbors:
        if HammingDistance(suffix, text) < d:
            for base in nucleotides:
                neighborhood.append(base+text)
        else:
            neighborhood.append(first+text)

    return neighborhood

In [5]:
def PatternToNumber(pattern):
    if pattern == "":
        return 0
    
    prefix = pattern[:-1]
    symbol = pattern[-1]
    symbol_num = {"A": 0, "C": 1, "G": 2, "T": 3}

    return 4 * PatternToNumber(prefix) + symbol_num[symbol]

In [6]:
def NumberToPattern(index, k):
    symbol_num = "ACGT"
    if k == 1:
        return symbol_num[index]

    prefix_index = index // 4
    remainder = index % 4

    symbol = symbol_num[remainder]
    prefix_pattern = NumberToPattern(prefix_index, k-1)

    return prefix_pattern + symbol

In [7]:
def ComputingFrequencies(text, k):

    # initialize array (length = 4**k) elements to zero 
    frequency_array = [0]*(4**k)

    bound = len(text) - k + 1
    for i in range(bound):
        pattern = text[i:i+k]
        j = PatternToNumber(pattern)
        frequency_array[j] += 1

    return frequency_array

In [8]:
def FreqWordsMismatch(text, k, d):
    
    frequent_patterns = []
    frequency_array = [0]*(4**k)
    close_array = [0]*(4**k)

    bound = len(text) - k + 1
    for i in range(bound):
        # Get neighbors (kmers with upto d mismatches)
        neighborhood = Neighbors(text[i:i+k], d)

        # Mark the patterns that are neighbors as 1 (close = True)
        for pattern in neighborhood:
            index = PatternToNumber(pattern)
            close_array[index] = 1

    # For those marked as True, count the num of occurences of pattern with 
    # at most d mismatches
    for i in range(4**k):
        if close_array[i] == 1:
            pattern = NumberToPattern(i, k)
            frequency_array[i] = ApproxPatternCount(pattern, text, d)
    
    # Find the pattern that has the highest occurrence (max of frequency_array)
    max_count = max(frequency_array)
    for i in range(4**k):
        if frequency_array[i] == max_count:
            pattern = NumberToPattern(i, k)
            frequent_patterns.append(pattern)
    
    return frequent_patterns

In [35]:
def FreqWordsMismatchRC(text, k, d):
    
    frequent_patterns = []
    frequency_array = [0]*(4**k)
    close_array = [0]*(4**k)

    bound = len(text) - k + 1
    for i in range(bound):
        # Get neighbors (kmers with upto d mismatches)
        neighborhood = Neighbors(text[i:i+k], d)

        # Mark the patterns that are neighbors as 1 (close = True)
        for pattern in neighborhood:
            index = PatternToNumber(pattern)
            close_array[index] = 1

            # Also for its reverse complement
            rc_pattern = ReverseComplement(pattern)
            rc_index = PatternToNumber(rc_pattern)
            close_array[rc_index] = 1

    # For those marked as True, count the num of occurences of pattern and
    # rc_pattern in text with at most d mismatches
    for i in range(4**k):
        if close_array[i] == 1:
            pattern = NumberToPattern(i, k)
            rc_pattern = ReverseComplement(pattern)
            frequency_array[i] = (ApproxPatternCount(pattern, text, d) +
                                  ApproxPatternCount(rc_pattern, text, d))
    
    # Find the pattern that has the highest occurrence (max of frequency_array)
    max_count = max(frequency_array)
    for i in range(4**k):
        if frequency_array[i] == max_count:
            pattern = NumberToPattern(i, k)
            frequent_patterns.append(pattern)
    
    return frequent_patterns, max_count

### Stepik Coding Exercise

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Salmonella_enterica.txt to Salmonella_enterica.txt


In [28]:
with open('/content/Salmonella_enterica.txt', "r") as f:
    contents = f.readlines()

    genome = ""
    for i in range(1, len(contents)):
        genome += contents[i]

    genome_long = genome.replace('\n', '')

    print(MinimumSkew(genome_long))

[3764856, 3764858]


In [37]:
minskew = 3764856
window = 1000

start, count  = FreqWordsMismatchRC(genome_long[minskew:minskew+window], 9, 1)
center, count = FreqWordsMismatchRC(genome_long[minskew-(window//2):minskew+(window//2)], 9, 1)
end, count    = FreqWordsMismatchRC(genome_long[minskew-window:minskew], 9, 1)

print(start, count)
print(center, count)
print(end, count)

['CCAGGATCC', 'GGATCCTGG'] 4
['TGTGGATAA', 'TTATCCACA'] 4
['AAAACCGAT', 'AAAATCTGG', 'AAACAATTG', 'AACACGATC', 'AACCAGATC', 'AAGCCGATC', 'ACGGCGCCG', 'ACTAATCAA', 'AGCTGATAA', 'AGTTGTTTG', 'ATCAATTGA', 'ATCGGTTTT', 'ATTATCAGC', 'CAAACAACT', 'CAATTGTTT', 'CAGATCGTC', 'CCAAACAAC', 'CCAGATTTT', 'CCCAAACAA', 'CCGGCGCCG', 'CGGCGCCGA', 'CGGCGCCGC', 'CGGCGCCGG', 'CGGCGCCGT', 'GACGATCTG', 'GATCGGCTT', 'GATCGTGTT', 'GATCTGGTT', 'GCCAATCAA', 'GCGGCGCCG', 'GCTGATAAT', 'GCTTCTTTA', 'GGCTGAAAA', 'GGTGGAAAA', 'GTTAAAAAA', 'GTTGTTTGG', 'TAAAGAAGC', 'TCAATTGAT', 'TCGGCGCCG', 'TTATCAGCT', 'TTGATTAGT', 'TTGATTGGC', 'TTGTTTGGG', 'TTTTCAGCC', 'TTTTCCACC', 'TTTTTTAAC'] 4
